In [19]:
# Importar librerias

import pandas as pd
import gzip
import json
import os
import pickle
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,median_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression






In [4]:
# Paso 1

train = pd.read_csv("/content/files/input/train_data.csv.zip",index_col=False,compression="zip")
test = pd.read_csv("/content/files/input/test_data.csv.zip",index_col=False,compression="zip")

train['Age'] = 2021 - train['Year']
test['Age'] = 2021 - test['Year']

train.drop(columns=['Year', 'Car_Name'], inplace=True)
test.drop(columns=['Year', 'Car_Name'], inplace=True)

print(train.head())




   Selling_Price  Present_Price  Driven_kms Fuel_Type Selling_type  \
0           7.40          8.500       15059    Petrol       Dealer   
1           4.00          4.600       30000    Petrol       Dealer   
2           0.50          0.826        6000    Petrol   Individual   
3           3.15          4.430       15000    Petrol       Dealer   
4           1.25          1.500       15000    Petrol   Individual   

  Transmission  Owner  Age  
0    Automatic      0    5  
1       Manual      0    8  
2       Manual      0   10  
3       Manual      0    5  
4       Manual      0    8  


In [6]:
# Paso 2.

x_train = train.drop(columns=['Selling_Price'])
y_train = train['Selling_Price']

x_test = test.drop(columns=['Selling_Price'])
y_test = test['Selling_Price']

# Mostrar un vistazo de las dimensiones de los datasets divididos
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")


x_train shape: (211, 7)
y_train shape: (211,)
x_test shape: (90, 7)
y_test shape: (90,)


In [32]:
# Paso 3
categorical_features = ['Fuel_Type', 'Selling_type', 'Transmission']
numeric_features= [col for col in x_train.columns if col not in categorical_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', MinMaxScaler(), numeric_features)
    ]
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('feature_selector', SelectKBest(score_func=f_regression, k='all')),
        ('regressor', LinearRegression())
    ]
)


In [35]:
# Paso 4.
param_grid = {
    'feature_selector__k': range(1, 15),
    'regressor__fit_intercept': [True, False],
}

model = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    refit=True
)

model.fit(x_train, y_train)

print(f"Mejores hiperparámetros: {model.best_params_}")




Mejores hiperparámetros: {'feature_selector__k': 11, 'regressor__fit_intercept': False}


In [36]:
# Paso 5.
models_dir = '/content/files/models'
os.makedirs(models_dir, exist_ok = True)

model_path = "/content/files/models/model.pkl.gz"

with gzip.open(model_path, "wb") as model_file:
    pickle.dump(model, model_file)

print(f"Modelo guardado en '{model_path}'")



Modelo guardado en '/content/files/models/model.pkl.gz'


In [25]:
# Paso 6
import json
import os
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,median_absolute_error

best_model=model.best_estimator_
# Calcular métricas para el conjunto de entrenamiento
y_train_pred = best_model.predict(x_train)
train_metrics = {
    'type': 'metrics',
    'dataset': 'train',
    'r2': r2_score(y_train, y_train_pred),
    'mse': mean_squared_error(y_train, y_train_pred),
    'mad': median_absolute_error(y_train, y_train_pred),
}

# Calcular métricas para el conjunto de prueba
y_test_pred = best_model.predict(x_test)
test_metrics = {
    'type': 'metrics',
    'dataset': 'test',
    'r2': r2_score(y_test, y_test_pred),
    'mse': mean_squared_error(y_test, y_test_pred),
    'mad': median_absolute_error(y_test, y_test_pred),
}

output_path = "/content/files/output/metrics.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as f:
    f.write(json.dumps(train_metrics) + '\n')
    f.write(json.dumps(test_metrics) + '\n')

print(f"Métricas guardadas en: {output_path}")





Métricas guardadas en: /content/files/output/metrics.json


In [40]:
model.score(x_train, y_train) < SCORES[0]

False